In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
titanic_data = pd.read_csv('data/titanic/train.csv')

In [3]:
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_rst = pd.read_excel('data\Rostest1.xlsx').rename(columns={'№ заявки': 'Номер'})
df_rst[['Номер', 'Статус', 'Сумма c НДС, ₽', 'Выездные работы',
       'Офис обслуживания', 'Платежные документы', 'Договор']] = df_rst[['Номер', 'Статус', 'Сумма c НДС, ₽', 'Выездные работы',
       'Офис обслуживания', 'Платежные документы', 'Договор']].apply(lambda x: x.str.strip(), axis=1)
df_rst['Сумма c НДС, ₽'] = df_rst['Сумма c НДС, ₽'].apply(lambda x: str(x).replace('-', '').replace('+', '').replace(' ', ''))
df_rst = df_rst.loc[df_rst['Номер'].notna()]
df_rst.head()

,Номер,Дата,Статус,"Сумма c НДС, ₽",Выездные работы,Офис обслуживания,Платежные документы,Договор
0,1000-087097,2024-12-03,В работе,954.00,NaN,Головное Ростест-Москва,—,№1000-150/2024/3618 от 01.08.2024
1,1000-087099,2024-12-03,В работе,12639.44,NaN,Головное Ростест-Москва,—,№1000-150/2024/3618 от 01.08.2024
2,1000-087093,2024-12-03,В работе,16464.00,NaN,Головное Ростест-Москва,Счет №1000-150/016313 от 26.08.2024,№1000-150/2024/3618 от 01.08.2024
4,1000-087091,2024-12-03,В работе,51444.00,NaN,Головное Ростест-Москва,Счет №1000-150/016313 от 26.08.2024,№1000-150/2024/3618 от 01.08.2024
6,1000-086863,2024-12-02,В работе,10800.00,NaN,Головное Ростест-Москва,Счет №1000-440/021885 от 02.12.2024,№1000-150/2024/699 от 27.02.2024


In [5]:
df_price = pd.read_excel('data\!Счета Ростест 2024.xlsx', skiprows=6).rename(columns={'№ заявки': 'Номер'})
df_price['Номер'] = df_price['Номер'].apply(lambda x: str(x).strip())
df_price.head()

,Номер,Работы,Кол-во,Счет,Без НДС,С НДС,Остаток,"Стоимость, Без НДС","Стоимость, с НДС",Закрытие (месяц),Закрытие (дата),Акт,СФ,Дата оплаты счета (месяц),№ пп,Справка
0,0600-004489,Газосигнализатор ИГС-98 Комета-М-4,4,1000-150/004958 от 25.03.2024,330000.0,396000.0,1145.42,6760.00,8112.00,май,NaN,NaN,NaN,апрель,NaN,. Прибор Газосигнализатор ИГС-98 Комета-М-4 в ...
1,0600-004499,Комплект дозиметров прямопоказывающих ДДГ-01Д,3,NaN,NaN,NaN,NaN,17400.00,20880.00,май,NaN,NaN,NaN,NaN,NaN,. Прибор Комплект дозиметров прямопоказывающих...
2,0600-004499,Комплект дозиметров ИД-1,10,NaN,NaN,NaN,NaN,5400.00,6480.00,май,NaN,NaN,NaN,NaN,NaN,. Прибор Комплект дозиметров ИД-1 в кол-ве 10ш...
3,0600-004499,"Дозиметр гамма-излучения ДКГ-03Д ""Грач""",1,NaN,NaN,NaN,NaN,5266.43,6319.72,май,NaN,NaN,NaN,NaN,NaN,". Прибор Дозиметр гамма-излучения ДКГ-03Д ""Гра..."
4,0600-004483,Преобразователь расхода вихреакустический Метр...,2,NaN,NaN,NaN,NaN,9900.00,11880.00,май,NaN,NaN,NaN,NaN,NaN,. Прибор Преобразователь расхода вихреакустиче...


In [6]:
df_price_new = df_price.groupby('Номер').agg({'Стоимость, с НДС': 'sum'})

In [7]:
df_merge = df_rst.merge(df_price_new, on='Номер', how='inner')[['Номер', 'Дата', 'Сумма c НДС, ₽', 'Стоимость, с НДС']]
df_merge['Разница'] = df_merge['Сумма c НДС, ₽'].astype('float') - df_merge['Стоимость, с НДС'].astype('float')
df_merge

,Номер,Дата,"Сумма c НДС, ₽","Стоимость, с НДС",Разница
0,0600-015769,2024-11-18,4464.00,2976.00,1488.0
1,0600-015768,2024-11-18,18240.00,18240.00,0.0
2,0600-015019,2024-11-01,18240.00,18240.00,0.0
3,0500-005869,2024-09-18,6448.80,6448.80,0.0
4,0600-012008,2024-09-10,29700.00,29700.00,0.0
5,0600-012004,2024-09-10,24156.27,24156.27,0.0
6,0600-012009,2024-09-10,23379.92,21879.92,1500.0
7,0600-010262,2024-08-12,669.14,669.14,0.0
8,0600-010261,2024-08-12,14940.00,12912.00,2028.0
9,0600-010267,2024-08-12,97740.00,97740.00,0.0


In [8]:
df_rst.loc[df_rst['Номер'] == '0600-015769']

,Номер,Дата,Статус,"Сумма c НДС, ₽",Выездные работы,Офис обслуживания,Платежные документы,Договор
80,0600-015769,2024-11-18,Выдано,4464.00,NaN,Сергиево-Посадский филиал,Счет №1000-150/011630 от 27.06.2024,№0100/2024/1818 от 19.03.2024


In [9]:
df_price.loc[df_price['Номер'] == '0600-015769']

,Номер,Работы,Кол-во,Счет,Без НДС,С НДС,Остаток,"Стоимость, Без НДС","Стоимость, с НДС",Закрытие (месяц),Закрытие (дата),Акт,СФ,Дата оплаты счета (месяц),№ пп,Справка
141,0600-015769,Термометр стеклянный TGL,2,NaN,NaN,NaN,NaN,2480.0,2976.0,ноябрь,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
round(1020769.2 - df_price['Стоимость, с НДС'].astype('float').sum(), 2)

4272.89

In [ ]:
# df_merge.to_excel('data/df_merge.xlsx')